In [17]:
fnames = pd.Series(io.l1a_filenames("mvn_iuv_l1a_APP2-orbit00256-mode0001-muvdark_20141115T222516_v02_s01", iterator=False, stage=True))
fnames.size

1

In [2]:
pd.options.display.max_columns = 100

In [38]:
from IPython.parallel import Client
c = Client()

In [39]:
dview = c.direct_view()
lview = c.load_balanced_view()

In [18]:
l1a = io.L1AReader(fnames[0])

In [19]:
l1a.img_header

SIMPLE  =                    T /Primary Header created by MWRFITS v1.11         
BITPIX  =                   32 /                                                
NAXIS   =                    2 /                                                
NAXIS1  =                  256 /                                                
NAXIS2  =                    7 /                                                
EXTEND  =                    T /Extensions may be present                       
COMMENT MAVEN IUVS Level 1A Data Product                                        
COMMENT IUVS Principal Investigator N. Schneider                                
COMMENT Laboratory for Atmospheric and Space Physics                            
COMMENT University of Colorado Boulder                                          
COMMENT 1234 Innovation Drive, Boulder CO 80303                                 
COMMENT MAVEN Mission scientific and model results are open to all.             
COMMENT Users should contact

In [24]:
l1a.Observation['PRODUCT_ID']

chararray(['mvn_iuv_l1a_APP2-orbit00256-mode0001-muvdark_20141115T222516_v02_s01'], 
      dtype='<U68')

In [13]:
l1a.n_integrations

2

In [15]:
l1a.n_dims

3

In [16]:
l1a.Integration

,TIMESTAMP,ET,UTC,MIRROR_DN,MIRROR_DEG,FOV_DEG,LYA_CENTROID,CASE_TEMP,DET_TEMP
0,4.740133e+08,4.740134e+08,2015/008 Jan 08 18:21:38.42850UTC,16087,8.64624,17.29248,1,15964,2600
1,4.740133e+08,4.740134e+08,2015/008 Jan 08 18:21:43.22849UTC,16087,8.64624,17.29248,1,15964,2604


## When I only scan for darks, len(fnames) < 10000, so I can keep the dataframes in memory.

### If that is a problem, don't return the dataframe at the end!

In [40]:
def process_fname(fname):
    import numpy as np
    l1a = io.L1AReader(fname)
    if l1a.n_dims ==2 and l1a.n_integrations > 1:
        return fname, 2
    main_header = io.get_header_df(l1a.hdulist[0]).convert_objects()
    integration = l1a.Integration.T.convert_objects()
    if main_header['NAXIS'] == 2:
        main_header['NAXIS3'] = np.nan
        avgtuple = None
    elif main_header['NAXIS'] == 3:
        avgtuple = (1,2)
    else:
        return fname, 1
    integration['mean'] = l1a.img.mean(axis=avgtuple)
    integration['median'] = np.median(l1a.img, axis=avgtuple)
    integration['std'] = l1a.img.std(axis=avgtuple)
    joined = integration.join(pd.concat([pd.DataFrame(main_header).T] * len(integration),
                                        ignore_index=True))
    for col in l1a.Observation.names[:-3]:
        val = l1a.Observation[col][0]
        if col == 'COLLECTION_ID':
            val = val[0]
        joined[col] = val
    joined = joined.convert_objects()
    savepath = io.save_to_hdf(joined, fname, 'l1a_dark_scans')
    return joined, 0

In [41]:
results = lview.map_async(process_fname, fnames)

In [42]:
from iuvs.multitools import nb_progress_display

In [43]:
nb_progress_display(results, fnames)

In [44]:
naxis_not_2_or_3 = []
good_ones = []
n_ints_larger1 = []
for res in results:
    if res[1] == 1:
        naxis_not_2_or_3.append(res[0])
    elif res[1] == 2:
        n_ints_larger1.append(res[0])
    else:
        good_ones.append(res[0])

In [45]:
len(n_ints_larger1)

6

In [46]:
len(naxis_not_2_or_3)

5

In [47]:
n_ints_larger1

['/maven_iuvs/stage/products/level1a/mvn_iuv_l1a_apoapse-orbit01243-muvdark_20150521T095658_v02_s01.fits.gz',
 '/maven_iuvs/stage/products/level1a/mvn_iuv_l1a_apoapse-orbit01236-muvdark_20150520T035347_v02_s01.fits.gz',
 '/maven_iuvs/stage/products/level1a/mvn_iuv_l1a_apoapse-orbit01236-fuvdark_20150520T035347_v02_s01.fits.gz',
 '/maven_iuvs/stage/products/level1a/mvn_iuv_l1a_apoapse-orbit01243-fuvdark_20150521T095658_v02_s01.fits.gz',
 '/maven_iuvs/stage/products/level1a/mvn_iuv_l1a_apoapse-orbit01243-fuvdark_20150521T104548_v02_s01.fits.gz',
 '/maven_iuvs/stage/products/level1a/mvn_iuv_l1a_apoapse-orbit01243-fuvdark_20150521T120507_v02_s01.fits.gz']

In [59]:
results_df = pd.concat(good_ones, ignore_index=True)

In [60]:
results_df.head()

,ANC_SVN_REVISION,BIN_PATTERN_INDEX,BIN_TBL,BITPIX,BLANK,BUNDLE_ID,CADENCE,CAPTURE,CASE_TEMP,CHANNEL,CODE_SVN_REVISION,COLLECTION_ID,DET_TEMP,DUTY_CYCLE,ET,EXTEND,FILENAME,FOV_DEG,GRATING_SELECT,INT_TIME,KEYHOLE_SELECT,LYA_CENTROID,MCP_VOLT,MIRROR_DEG,MIRROR_DN,MIR_DEG,MISSION_PHASE,NAXIS,NAXIS1,NAXIS2,NAXIS3,N_FILL,OBSERVATION_TYPE,OBS_ID,ORBIT_NUMBER,ORBIT_SEGMENT,PROCESS,PRODUCT_CREATION_DATE,PRODUCT_ID,SIMPLE,SOLAR_LONGITUDE,SPA_OFS,SPA_SIZE,SPE_OFS,SPE_SIZE,SW_VER,TARGET_NAME,TIMESTAMP,UTC,XUV,mean,median,std
0,929,LINEAR linear_0004,"LINEAR 5,6 linear_0004",32,-1,raw,4.8,2014/338 Dec 04 02:03:22.79468UTC,15972,FUV,929,limb,2594,1.000000,4.709307e+08,True,mvn_iuv_l1a_periapse-orbit00351-fuvdark_201412...,23.373413,LOWRES,4.2,NEITHER,1,-1.83,11.686707,17194,11.689,PRIME,2,256,7,NaN,0,CDS,4,351,0,2015/108 Apr 18 18:28:06.00000UTC,2015/108 Apr 18 18:28:06.00000UTC,mvn_iuv_l1a_periapse-orbit00351-fuvdark_201412...,True,245.792343,89,115,0,4,929,MARS,4.709306e+08,2014/338 Dec 04 02:03:22.79468UTC,FUV,1024.759487,993.5,242.897055
1,929,LINEAR linear_0006,"LINEAR 5,6 linear_0006",32,-1,raw,4.8,2015/008 Jan 08 18:21:38.42850UTC,15964,FUV,929,limb,2600,1.000000,4.740134e+08,True,mvn_iuv_l1a_periapse-orbit00538-fuvdark_201501...,17.292480,LOWRES,4.2,NEITHER,1,-1.83,8.646240,16087,8.649,PRIME,3,256,7,2,0,CDS,6,538,0,2015/108 Apr 18 22:22:19.00000UTC,2015/108 Apr 18 22:22:19.00000UTC,mvn_iuv_l1a_periapse-orbit00538-fuvdark_201501...,True,268.399689,89,115,0,4,929,MARS,4.740133e+08,2015/008 Jan 08 18:21:38.42850UTC,FUV,1098.962612,1063.0,263.214653
2,929,LINEAR linear_0006,"LINEAR 5,6 linear_0006",32,-1,raw,4.8,2015/008 Jan 08 18:21:38.42850UTC,15964,FUV,929,limb,2604,1.000000,4.740134e+08,True,mvn_iuv_l1a_periapse-orbit00538-fuvdark_201501...,17.292480,LOWRES,4.2,NEITHER,1,-1.83,8.646240,16087,8.649,PRIME,3,256,7,2,0,CDS,6,538,0,2015/108 Apr 18 22:22:19.00000UTC,2015/108 Apr 18 22:22:19.00000UTC,mvn_iuv_l1a_periapse-orbit00538-fuvdark_201501...,True,268.399689,89,115,0,4,929,MARS,4.740133e+08,2015/008 Jan 08 18:21:43.22849UTC,FUV,1012.230469,974.0,239.967113
3,929,LINEAR linear_0004,"LINEAR 15,16 linear_0004",32,-1,raw,15.0,2014/336 Dec 02 05:40:46.04215UTC,15928,FUV,929,disk,2589,1.000000,4.707709e+08,True,mvn_iuv_l1a_apoapse-orbit00341-fuvdark_2014120...,65.797119,LOWRES,14.4,NEITHER,1,-1.83,32.898560,24917,32.901,PRIME,2,184,10,NaN,0,CDS,8196,341,0,2015/108 Apr 18 18:13:25.00000UTC,2015/108 Apr 18 18:13:25.00000UTC,mvn_iuv_l1a_apoapse-orbit00341-fuvdark_2014120...,True,244.619370,89,80,65,4,929,MARS,4.707708e+08,2014/336 Dec 02 05:40:46.04215UTC,FUV,1469.102174,1393.0,398.650876
4,929,LINEAR CSS_S4,"LINEAR 17,18 CSS_S4",32,-1,raw,15.0,2014/291 Oct 18 17:54:10.74999UTC,15929,MUV,929:931M,disk,2572,0.200855,4.669269e+08,True,mvn_iuv_l1a_apoapse-orbit00109-muvdark_2014101...,65.797119,LOWRES,14.4,NEITHER,-1,-1.83,32.898560,24917,32.901,PRIME,2,30,10,NaN,0,CDS,2001,109,0,2015/115 Apr 25 03:58:15.00000UTC,2015/115 Apr 25 03:58:15.00000UTC,mvn_iuv_l1a_apoapse-orbit00109-muvdark_2014101...,True,216.717957,101,80,2,34,929:931M,,4.669268e+08,2014/291 Oct 18 17:54:10.74999UTC,MUV,15498.593333,15320.0,1563.283871


In [61]:
results_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18826 entries, 0 to 18825
Data columns (total 53 columns):
ANC_SVN_REVISION         18826 non-null object
BIN_PATTERN_INDEX        18826 non-null object
BIN_TBL                  18826 non-null object
BITPIX                   18826 non-null int64
BLANK                    18826 non-null int64
BUNDLE_ID                18826 non-null object
CADENCE                  18826 non-null float64
CAPTURE                  18826 non-null object
CASE_TEMP                18826 non-null float64
CHANNEL                  18826 non-null object
CODE_SVN_REVISION        18826 non-null object
COLLECTION_ID            18826 non-null object
DET_TEMP                 18826 non-null float64
DUTY_CYCLE               18826 non-null float64
ET                       18826 non-null float64
EXTEND                   18826 non-null bool
FILENAME                 18826 non-null object
FOV_DEG                  18826 non-null float64
GRATING_SELECT           18826 non-null obj

In [62]:
results_df.loc[results_df.ANC_SVN_REVISION == '', 'ANC_SVN_REVISION'] = 0

In [63]:
results_df = results_df.convert_objects(convert_numeric=True)

In [64]:
results_df.to_hdf('/home/klay6683/output/l1a_dark_scans/results_df.h5', 'df')

### For longer scans when I can't keep everything in memory

In [10]:
import glob
h5fnames = glob.glob("/home/klay6683/output/l1a_dark_scans/*.h5")

In [11]:
len(h5fnames)

40421

In [12]:
def chunker(seq, size):
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))

In [13]:
dfs = []
for i,chunk in enumerate(chunker(h5fnames, 200)):
    print("Chunk {}".format(i))
    frames = []
    for fname in chunk:
        frames.append(pd.read_hdf(fname, 'df'))
    dfs.append(pd.concat(frames, ignore_index=True))

Chunk 0
Chunk 1
Chunk 2
Chunk 3
Chunk 4
Chunk 5
Chunk 6
Chunk 7
Chunk 8
Chunk 9
Chunk 10
Chunk 11
Chunk 12
Chunk 13
Chunk 14
Chunk 15
Chunk 16
Chunk 17
Chunk 18
Chunk 19
Chunk 20
Chunk 21
Chunk 22
Chunk 23
Chunk 24
Chunk 25
Chunk 26
Chunk 27
Chunk 28
Chunk 29
Chunk 30
Chunk 31
Chunk 32
Chunk 33
Chunk 34
Chunk 35
Chunk 36
Chunk 37
Chunk 38
Chunk 39
Chunk 40
Chunk 41
Chunk 42
Chunk 43
Chunk 44
Chunk 45
Chunk 46
Chunk 47
Chunk 48
Chunk 49
Chunk 50
Chunk 51
Chunk 52
Chunk 53
Chunk 54
Chunk 55
Chunk 56
Chunk 57
Chunk 58
Chunk 59
Chunk 60
Chunk 61
Chunk 62
Chunk 63
Chunk 64
Chunk 65
Chunk 66
Chunk 67
Chunk 68
Chunk 69
Chunk 70
Chunk 71
Chunk 72
Chunk 73
Chunk 74
Chunk 75
Chunk 76
Chunk 77
Chunk 78
Chunk 79
Chunk 80
Chunk 81
Chunk 82
Chunk 83
Chunk 84
Chunk 85
Chunk 86
Chunk 87
Chunk 88
Chunk 89
Chunk 90
Chunk 91
Chunk 92
Chunk 93
Chunk 94
Chunk 95
Chunk 96
Chunk 97
Chunk 98
Chunk 99
Chunk 100
Chunk 101
Chunk 102
Chunk 103
Chunk 104
Chunk 105
Chunk 106
Chunk 107
Chunk 108
Chunk 109
Chunk 110


In [14]:
superdf = pd.concat(dfs, ignore_index=True)

In [15]:
superdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40421 entries, 0 to 40420
Data columns (total 63 columns):
AVERAGE               40421 non-null float64
BIN_SHIFT             40421 non-null int64
BIN_SHIFT_DIR         40421 non-null int64
BIN_TBL               40421 non-null object
BIN_TYPE              40421 non-null object
BIN_X_ROW             40421 non-null int64
BIN_Y_ROW             40421 non-null int64
BITPIX                40421 non-null int64
BLANK                 40421 non-null int64
BSCALE                494 non-null float64
BZERO                 494 non-null float64
CADENCE               40421 non-null float64
CAPTURE               40421 non-null object
CASE_TEMP             39927 non-null float64
CHECKSUM              40421 non-null float64
DATA_COMPRESSION      40421 non-null int64
DET_TEMP              39927 non-null float64
EXTEND                40421 non-null bool
FILENAME              40421 non-null object
FUV_BAD_PIXEL_MASK    40421 non-null int64
IMAGE_NUMBER      

In [18]:
from iuvs import calib

In [19]:
superdf.DET_TEMP = superdf.DET_TEMP.map(calib.convert_det_temp_to_C)
superdf.CASE_TEMP = superdf.CASE_TEMP.map(calib.convert_case_temp_to_C)

In [20]:
superdf.to_hdf('/home/klay6683/output/l1a_summary.h5','df')

In [21]:
meta.produce_summary_txt(superdf, 'l1a_summary.txt')